In [1]:
using CSV, MLJ, DataFrames, RDatasets, DecisionTree, CategoricalArrays

In [2]:
data = CSV.read("D:/Master/S7/R-Julia/Projet/CO2 Emissions_Canada.csv", DataFrames.DataFrame)
rename!(data, :"CO2 Emissions(g/km)"=> "CO2")
rename!(data, :"Vehicle Class"=>"Class")
rename!(data, :"Engine Size(L)"=>"Engine")
rename!(data, :"Fuel Type"=>"Fuel")
rename!(data, :"Fuel Consumption City (L/100 km)"=>"City")
rename!(data, :"Fuel Consumption Hwy (L/100 km)"=>"Hwy")
rename!(data, :"Fuel Consumption Comb (L/100 km)"=>"Comb")
rename!(data, :"Fuel Consumption Comb (mpg)"=>"mpg")

names(data)

12-element Vector{String}:
 "Make"
 "Model"
 "Class"
 "Engine"
 "Cylinders"
 "Transmission"
 "Fuel"
 "City"
 "Hwy"
 "Comb"
 "mpg"
 "CO2"

In [3]:
Qualitatives = ["Make","Model","Transmission","Fuel","Class"]
Quantitatives=["Engine","Cylinders","City","Hwy","Comb"]#,"mpg"]
Target = ["CO2"]
Features = append!(Qualitatives, Quantitatives)

10-element Vector{String}:
 "Make"
 "Model"
 "Transmission"
 "Fuel"
 "Class"
 "Engine"
 "Cylinders"
 "City"
 "Hwy"
 "Comb"

In [4]:
y = data.CO2
X = select(data, Not(:CO2))

Row,Make,Model,Class,Engine,Cylinders,Transmission,Fuel,City,Hwy,Comb,mpg
,String15,String,String31,Float64,Int64,String7,String1,Float64,Float64,Float64,Int64
1,ACURA,ILX,COMPACT,2.0,4,AS5,Z,9.9,6.7,8.5,33
2,ACURA,ILX,COMPACT,2.4,4,M6,Z,11.2,7.7,9.6,29
3,ACURA,ILX HYBRID,COMPACT,1.5,4,AV7,Z,6.0,5.8,5.9,48
4,ACURA,MDX 4WD,SUV - SMALL,3.5,6,AS6,Z,12.7,9.1,11.1,25
5,ACURA,RDX AWD,SUV - SMALL,3.5,6,AS6,Z,12.1,8.7,10.6,27
6,ACURA,RLX,MID-SIZE,3.5,6,AS6,Z,11.9,7.7,10.0,28
7,ACURA,TL,MID-SIZE,3.5,6,AS6,Z,11.8,8.1,10.1,28
8,ACURA,TL AWD,MID-SIZE,3.7,6,AS6,Z,12.8,9.0,11.1,25
9,ACURA,TL AWD,MID-SIZE,3.7,6,M6,Z,13.4,9.5,11.6,24


In [5]:
train, test = partition(eachindex(y), 0.7, shuffle=true, rng=424);
train[1:3]

3-element Vector{Int64}:
 2163
 1783
 5638

In [6]:
Xquali =  select(X,Qualitatives)
Xquantitatives = select(X,Quantitatives)

Row,Engine,Cylinders,City,Hwy,Comb
,Float64,Int64,Float64,Float64,Float64
1,2.0,4,9.9,6.7,8.5
2,2.4,4,11.2,7.7,9.6
3,1.5,4,6.0,5.8,5.9
4,3.5,6,12.7,9.1,11.1
5,3.5,6,12.1,8.7,10.6
6,3.5,6,11.9,7.7,10.0
7,3.5,6,11.8,8.1,10.1
8,3.7,6,12.8,9.0,11.1
9,3.7,6,13.4,9.5,11.6


In [7]:
XTrain = X[train,:]
ycut=cut(y,20)
XTest=X[test,:]
yTrain = y[train]

5170-element Vector{Int64}:
 179
 269
 175
 210
 330
 358
 225
 285
 320
 347
   ⋮
 214
 235
 252
 196
 264
 269
 300
 179
 373

In [8]:
using CategoricalArrays
Make2=convert(Vector, data[!,"Make"])
CategoricalArray(Make2)
Model2=convert(Vector, data[!,"Model"])
CategoricalArray(Model2)
Transmission2=convert(Vector, data[!,"Transmission"])
CategoricalArray(Transmission2)
Fuel_type2=convert(Vector, data[!,"Fuel"])
CategoricalArray(Fuel_type2)
Vehicle_class2=convert(Vector, data[!,"Class"])
CategoricalArray(Vehicle_class2)


using DataFrames
Xquali2=DataFrame(Make=categorical(Make2),Model=categorical(Model2),Transmission=categorical(Transmission2),Fuel_type=categorical(Fuel_type2),Vehicle_class=categorical(Vehicle_class2))

hot = OneHotEncoder()
mach = MLJ.fit!(machine(hot, Xquali2))
W = MLJ.transform(mach, Xquali2)

┌ Info: Training machine(OneHotEncoder(features = Symbol[], …), …).
└ @ MLJBase C:\Users\33750\.julia\packages\MLJBase\ByFwA\src\machines.jl:492
┌ Info: Spawning 42 sub-features to one-hot encode feature :Make.
└ @ MLJModels C:\Users\33750\.julia\packages\MLJModels\L1gIK\src\builtins\Transformers.jl:878


┌ Info: Spawning 2053 sub-features to one-hot encode feature :Model.
└ @ MLJModels C:\Users\33750\.julia\packages\MLJModels\L1gIK\src\builtins\Transformers.jl:878


┌ Info: Spawning 27 sub-features to one-hot encode feature :Transmission.
└ @ MLJModels C:\Users\33750\.julia\packages\MLJModels\L1gIK\src\builtins\Transformers.jl:878


┌ Info: Spawning 5 sub-features to one-hot encode feature :Fuel_type.
└ @ MLJModels C:\Users\33750\.julia\packages\MLJModels\L1gIK\src\builtins\Transformers.jl:878


┌ Info: Spawning 16 sub-features to one-hot encode feature :Vehicle_class.
└ @ MLJModels C:\Users\33750\.julia\packages\MLJModels\L1gIK\src\builtins\Transformers.jl:878


Row,Make__ACURA,Make__ALFA ROMEO,Make__ASTON MARTIN,Make__AUDI,Make__BENTLEY,Make__BMW,Make__BUGATTI,Make__BUICK,Make__CADILLAC,Make__CHEVROLET,Make__CHRYSLER,Make__DODGE,Make__FIAT,Make__FORD,Make__GENESIS,Make__GMC,Make__HONDA,Make__HYUNDAI,Make__INFINITI,Make__JAGUAR,Make__JEEP,Make__KIA,Make__LAMBORGHINI,Make__LAND ROVER,Make__LEXUS,Make__LINCOLN,Make__MASERATI,Make__MAZDA,Make__MERCEDES-BENZ,Make__MINI,Make__MITSUBISHI,Make__NISSAN,Make__PORSCHE,Make__RAM,Make__ROLLS-ROYCE,Make__SCION,Make__SMART,Make__SRT,Make__SUBARU,Make__TOYOTA,Make__VOLKSWAGEN,Make__VOLVO,Model__124 SPIDER,Model__124 Spider,Model__1500,Model__1500 (MDS),Model__1500 4X4,Model__1500 4X4 (MDS),Model__1500 4X4 DIESEL,Model__1500 4X4 ECODIESEL,Model__1500 4X4 EcoDiesel,Model__1500 4X4 FFV,Model__1500 4X4 eTorque,Model__1500 Classic,Model__1500 Classic 4X4,Model__1500 Classic 4X4 EcoDiesel,Model__1500 Classic 4X4 FFV,Model__1500 Classic EcoDiesel,Model__1500 Classic FFV,Model__1500 DIESEL,Model__1500 ECODIESEL,Model__1500 ECODIESEL HFE,Model__1500 EcoDiesel,Model__1500 FFV,Model__1500 HFE,Model__1500 HFE eTorque,Model__1500 eTorque,Model__200,Model__200 AWD,Model__200 AWD FFV,Model__200 CONVERTIBLE,Model__200 CONVERTIBLE FFV,Model__200 FFV,Model__200 SEDAN,Model__200 SEDAN FFV,Model__228i,Model__228i CABRIOLET,Model__228i COUPE,Model__228i xDRIVE,Model__228i xDRIVE CABRIOLET,Model__228i xDRIVE COUPE,Model__230i CABRIOLET,Model__230i COUPE,Model__230i Cabriolet,Model__230i Coupe,Model__230i xDRIVE CABRIOLET,Model__230i xDRIVE COUPE,Model__230i xDrive Cabriolet,Model__230i xDrive Coupe,Model__300,Model__300 (MDS),Model__300 AWD,Model__300 AWD (MDS),Model__300 AWD FFV,Model__300 FFV,Model__300 SRT (MDS),Model__320i,Model__320i SEDAN,Model__320i xDRIVE,Model__320i xDRIVE SEDAN,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [32]:
X2=hcat(W,Xquali2)

Row,Make__ACURA,Make__ALFA ROMEO,Make__ASTON MARTIN,Make__AUDI,Make__BENTLEY,Make__BMW,Make__BUGATTI,Make__BUICK,Make__CADILLAC,Make__CHEVROLET,Make__CHRYSLER,Make__DODGE,Make__FIAT,Make__FORD,Make__GENESIS,Make__GMC,Make__HONDA,Make__HYUNDAI,Make__INFINITI,Make__JAGUAR,Make__JEEP,Make__KIA,Make__LAMBORGHINI,Make__LAND ROVER,Make__LEXUS,Make__LINCOLN,Make__MASERATI,Make__MAZDA,Make__MERCEDES-BENZ,Make__MINI,Make__MITSUBISHI,Make__NISSAN,Make__PORSCHE,Make__RAM,Make__ROLLS-ROYCE,Make__SCION,Make__SMART,Make__SRT,Make__SUBARU,Make__TOYOTA,Make__VOLKSWAGEN,Make__VOLVO,Model__124 SPIDER,Model__124 Spider,Model__1500,Model__1500 (MDS),Model__1500 4X4,Model__1500 4X4 (MDS),Model__1500 4X4 DIESEL,Model__1500 4X4 ECODIESEL,Model__1500 4X4 EcoDiesel,Model__1500 4X4 FFV,Model__1500 4X4 eTorque,Model__1500 Classic,Model__1500 Classic 4X4,Model__1500 Classic 4X4 EcoDiesel,Model__1500 Classic 4X4 FFV,Model__1500 Classic EcoDiesel,Model__1500 Classic FFV,Model__1500 DIESEL,Model__1500 ECODIESEL,Model__1500 ECODIESEL HFE,Model__1500 EcoDiesel,Model__1500 FFV,Model__1500 HFE,Model__1500 HFE eTorque,Model__1500 eTorque,Model__200,Model__200 AWD,Model__200 AWD FFV,Model__200 CONVERTIBLE,Model__200 CONVERTIBLE FFV,Model__200 FFV,Model__200 SEDAN,Model__200 SEDAN FFV,Model__228i,Model__228i CABRIOLET,Model__228i COUPE,Model__228i xDRIVE,Model__228i xDRIVE CABRIOLET,Model__228i xDRIVE COUPE,Model__230i CABRIOLET,Model__230i COUPE,Model__230i Cabriolet,Model__230i Coupe,Model__230i xDRIVE CABRIOLET,Model__230i xDRIVE COUPE,Model__230i xDrive Cabriolet,Model__230i xDrive Coupe,Model__300,Model__300 (MDS),Model__300 AWD,Model__300 AWD (MDS),Model__300 AWD FFV,Model__300 FFV,Model__300 SRT (MDS),Model__320i,Model__320i SEDAN,Model__320i xDRIVE,Model__320i xDRIVE SEDAN,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [26]:
fuelType = data[!,"Fuel"]
function f(x)
    if x == "E" return 0
    end
    return 1
end
fuelType=f.(fuelType)
X3=hcat(fuelType,X)

Row,x1,Make,Model,Class,Engine,Cylinders,Transmission,Fuel,City,Hwy,Comb,mpg
,Int64,String15,String,String31,Float64,Int64,String7,String1,Float64,Float64,Float64,Int64
1,1,ACURA,ILX,COMPACT,2.0,4,AS5,Z,9.9,6.7,8.5,33
2,1,ACURA,ILX,COMPACT,2.4,4,M6,Z,11.2,7.7,9.6,29
3,1,ACURA,ILX HYBRID,COMPACT,1.5,4,AV7,Z,6.0,5.8,5.9,48
4,1,ACURA,MDX 4WD,SUV - SMALL,3.5,6,AS6,Z,12.7,9.1,11.1,25
5,1,ACURA,RDX AWD,SUV - SMALL,3.5,6,AS6,Z,12.1,8.7,10.6,27
6,1,ACURA,RLX,MID-SIZE,3.5,6,AS6,Z,11.9,7.7,10.0,28
7,1,ACURA,TL,MID-SIZE,3.5,6,AS6,Z,11.8,8.1,10.1,28
8,1,ACURA,TL AWD,MID-SIZE,3.7,6,AS6,Z,12.8,9.0,11.1,25
9,1,ACURA,TL AWD,MID-SIZE,3.7,6,M6,Z,13.4,9.5,11.6,24


In [34]:
MyDecisionTree = @load DecisionTreeClassifier pkg = DecisionTree
dc = MyDecisionTree()
#dc.max_depth = 15
dc_mach = machine(dc, X[train,:], ycut[train])
MLJ.fit!(dc_mach)

import MLJDecisionTreeInterface ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main C:\Users\33750\.julia\packages\MLJModels\L1gIK\src\loading.jl:159
┌ Warning: The number and/or types of data arguments do not match what the specified model
│ supports. Suppress this type check by specifying `scitype_check_level=0`.
│ 
│ Run `@doc DecisionTree.DecisionTreeClassifier` to learn more about your model's requirements.
│ 
│ Commonly, but non exclusively, supervised models are constructed using the syntax
│ `machine(model, X, y)` or `machine(model, X, y, w)` while most other models are
│ constructed with `machine(model, X)`.  Here `X` are features, `y` a target, and `w`
│ sample or class weights.
│ 
│ In general, data in `machine(model, data...)` is expected to satisfy
│ 
│     scitype(data) <: MLJ.fit_data_scitype(model)
│ 
│ In the present case:
│ 
│ scitype(data) = Tuple{Table{Union{AbstractVector{Continuous}, AbstractVector{Count}, AbstractVector{Textual}}}, AbstractVector{OrderedFactor{20}}}
│ 
│ fit_data_scity

┌ Info: Training machine(DecisionTreeClassifier(max_depth = -1, …), …).
└ @ MLJBase C:\Users\33750\.julia\packages\MLJBase\ByFwA\src\machines.jl:492


trained Machine; caches model-specific representations of data
  model: DecisionTreeClassifier(max_depth = -1, …)
  args: 
    1:	Source @306 ⏎ Table{Union{AbstractVector{Continuous}, AbstractVector{Count}, AbstractVector{Textual}}}
    2:	Source @152 ⏎ AbstractVector{OrderedFactor{20}}


In [35]:
ypred = MLJ.predict_mode(dc_mach, X[test,:])

2215-element CategoricalArray{String,1,UInt32}:
 "Q3: [181.0, 191.0)"
 "Q16: [288.0, 298.0)"
 "Q13: [260.0, 269.0)"
 "Q19: [327.0, 354.0)"
 "Q3: [181.0, 191.0)"
 "Q16: [288.0, 298.0)"
 "Q6: [208.0, 216.0)"
 "Q16: [288.0, 298.0)"
 "Q16: [288.0, 298.0)"
 "Q12: [253.0, 260.0)"
 ⋮
 "Q14: [269.0, 278.0)"
 "Q19: [327.0, 354.0)"
 "Q18: [310.39999999999964, 327.0)"
 "Q13: [260.0, 269.0)"
 "Q20: [354.0, 522.0]"
 "Q17: [298.0, 310.39999999999964)"
 "Q10: [238.0, 246.0)"
 "Q8: [223.0, 230.0)"
 "Q9: [230.0, 238.0)"

In [36]:
println(accuracy(ypred, ycut[test]))
correct_classes = count(ycut[test] .== ypred)
# sans séparateur ethanol/ autres : 0.8492099322799097 1881
# avec : 0.8478555304740406 1878 -> pas besoin de le mettre

0.8455981941309255


1873

In [37]:
feature_importances(dc_mach)

11-element Vector{Pair{Symbol, Float64}}:
         :Comb => 0.7917666924695433
         :Fuel => 0.06404446720121028
          :mpg => 0.03214425983689642
        :Model => 0.024567690665198483
          :Hwy => 0.019991372712746116
         :City => 0.01850471828474936
 :Transmission => 0.016572053293334214
         :Make => 0.013475038557537004
       :Engine => 0.009275347233713805
        :Class => 0.00895432489325063
    :Cylinders => 0.0007040348518203548

In [39]:
MyDecisionTree2 = @load DecisionTreeRegressor pkg = DecisionTree
dcr = MyDecisionTree2()
dcr_mach = machine(dcr, X3[train,:], y[train])
MLJ.fit!(dcr_mach)

import MLJDecisionTreeInterface ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main C:\Users\33750\.julia\packages\MLJModels\L1gIK\src\loading.jl:159
┌ Warning: The number and/or types of data arguments do not match what the specified model
│ supports. Suppress this type check by specifying `scitype_check_level=0`.
│ 
│ Run `@doc DecisionTree.DecisionTreeRegressor` to learn more about your model's requirements.
│ 
│ Commonly, but non exclusively, supervised models are constructed using the syntax
│ `machine(model, X, y)` or `machine(model, X, y, w)` while most other models are
│ constructed with `machine(model, X)`.  Here `X` are features, `y` a target, and `w`
│ sample or class weights.
│ 
│ In general, data in `machine(model, data...)` is expected to satisfy
│ 
│     scitype(data) <: MLJ.fit_data_scitype(model)
│ 
│ In the present case:
│ 
│ scitype(data) = Tuple{Table{Union{AbstractVector{Continuous}, AbstractVector{Count}, AbstractVector{Textual}}}, AbstractVector{Count}}
│ 
│ fit_data_scitype(model) = T

trained Machine; caches model-specific representations of data
  model: DecisionTreeRegressor(max_depth = -1, …)
  args: 
    1:	Source @034 ⏎ Table{Union{AbstractVector{Continuous}, AbstractVector{Count}, AbstractVector{Textual}}}
    2:	Source @200 ⏎ AbstractVector{Count}


In [40]:
ypredr = MLJ.predict_mode(dcr_mach, X3[test,:])

2215-element Vector{Float64}:
 186.875
 295.875
 264.8333333333333
 345.55555555555554
 192.14285714285714
 291.42857142857144
 204.66666666666666
 296.2
 296.2
 253.0
   ⋮
 270.14285714285717
 327.125
 320.0
 264.42857142857144
 400.25
 304.8
 236.0
 225.0
 235.85714285714286

In [41]:
m=0
min=10
max=0
mini=0
maxi=0
v=1:length(test)
for i in v
    n=abs(y[test][i]-ypredr[i])
    if (n>max) 
        max = n
        maxi=i
     end
    if (n<min) 
        min = n
        mini = i
     end
    m= m+n
end
println("distance moyenne : ",m/length(test))
println("distance max ",max , " i = ", maxi)
println("distance min", min, " i = ", mini)

distance moyenne : 2.328443089391169
distance max 82.375 i = 38
distance min0.0 i = 10


In [ ]:
""" Avec séparateur ethanol / autres :
distance moyenne : 2.3221264890113633
distance max 82.375 i = 38
distance min0.0 i = 10
0.8126410835214447
4.64
Mieux avec 
"""

In [42]:
c=0
for i in v
    n=abs(y[test][i]-ypredr[i])
    if (n>13) 
        c=c+1
    end

end
println(c/length(test)*100)

0.8126410835214447

In [43]:
round(rms(ypredr, y[test]), sigdigits=3)

4.65

In [44]:
feature_importances(dcr_mach)

12-element Vector{Pair{Symbol, Float64}}:
         :Comb => 0.9468955190288346
           :x1 => 0.04072398439516061
         :Fuel => 0.008915364610652754
          :mpg => 0.002059993780793983
         :Make => 0.0002781544095387943
         :City => 0.0002618791574918742
          :Hwy => 0.0002515058094895561
       :Engine => 0.0002043988304768994
        :Model => 0.0001975699501606361
 :Transmission => 0.00011767092304548185
        :Class => 6.361214981587146e-5
    :Cylinders => 3.0346954538855293e-5

In [45]:
function predire_int(Marque, Modele, Class, Taille_moteur, Cylindres, Transmission,Fuel ,City,Hwy,Comb)
    mpg = Comb *235.214583
    donnees = DataFrame(Make=Marque, Model=Modele, Class=Class, Engine=Taille_moteur, Cylinders=Cylindres, Transmission=Transmission,Fuel=Fuel,City=City, Hwy=Hwy, Comb=Comb, mpg=mpg )
    s = string.(MLJ.predict_mode(dc_mach, donnees))
    s = s[1]
        return "L'intevalle est " * s[4:length(s)-1] * "]"
    
end

predire_int (generic function with 1 method)

In [46]:
predire_int("ACURA","ILX","COMPACT",2,4,"AS5","Z",9.9,6.7,8.5)

"L'intevalle est  [191.0, 199.0]"

In [47]:
function predire_val(Marque, Modele, Class, Taille_moteur, Cylindres, Transmission,Fuel ,City,Hwy,Comb)
    mpg = Comb *235.214583
    fuel =  f(Fuel)  
    donnees = DataFrame(Make=Marque, Model=Modele, Class=Class, Engine=Taille_moteur, Cylinders=Cylindres, Transmission=Transmission,Fuel=Fuel,City=City, Hwy=Hwy, Comb=Comb, mpg=mpg )
    x1 = DataFrame(fuel=fuel)
    donnees=hcat(x1, donnees)
    s = string.(MLJ.predict_mode(dcr_mach, donnees))
    s = s[1]
    return "La prédiction est " * s
    
end

predire_val (generic function with 1 method)

In [48]:
predire_val("ACURA","ILX","COMPACT",2,4,"AS5","Z",9.9,6.7,8.5)

"La prédiction est 198.0"

In [50]:
using JLD2
save_object("arbre_classif.jld2", dc_mach)
save_object("arbre_predict.jdl2", dcr_mach)